**CSI 4106 Introduction to Artificial Intelligence** <br/>
*Assignment 3: Neural Networks*

# Identification

Name: Ken Chan Thim <br/>
Student Number: 300208086

Name: <br/>
Student Number:

In [50]:
# Code cell


## 1. Analyse exploratoire

### Chargement de l'ensemble de données

Un ensemble de données a été créé pour ce devoir. Il est disponible sur un dépôt GitHub public :

- [github.com/turcotte/csi4106-f24/tree/main/assignments-data/a3](https://github.com/turcotte/csi4106-f24/tree/main/assignments-data/a3)

Vous devez accéder à l'ensemble de données et le lire directement à partir de ce dépôt GitHub dans votre notebook Jupyter.

Utilisez cette cellule de code pour vos directives `import` et autres initialisations.

In [51]:
# Code cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

1. **Charger l'ensemble de données** :

    - Écrivez du code pour charger les trois ensembles de données.


In [ ]:
# Code cell
df_train = pd.read_csv("https://raw.githubusercontent.com/turcotte/csi4106-f24/refs/heads/main/assignments-data/a3/cb513_train.csv", header=None)
df_test = pd.read_csv("https://raw.githubusercontent.com/turcotte/csi4106-f24/refs/heads/main/assignments-data/a3/cb513_test.csv", header=None)
df_valid = pd.read_csv("https://raw.githubusercontent.com/turcotte/csi4106-f24/refs/heads/main/assignments-data/a3/cb513_valid.csv", header=None)

df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

### Prétraitement des données

2. **Brasser les lignes** :

    - Étant donné que les exemples sont générés en faisant glisser une fenêtre sur chaque séquence de protéines, la plupart des exemples adjacents proviennent de la même protéine et partagent 20 positions. Pour atténuer l'impact potentiel négatif sur l'entraînement du modèle, la première étape consiste à brasser (*shuffle*) les **lignes** de la matrice de données.


In [ ]:
# Code cell

# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_valid = df_valid.sample(frac=1).reset_index(drop=True)

print(f"After shuffling: \n{df_train.head()}")

3. **Mise à l'échelle des caractéristiques numériques** :

    - Étant donné que les 462 caractéristiques sont des proportions représentées par des valeurs comprises entre 0 et 1, la mise à l'échelle peut ne pas être nécessaire. Dans nos évaluations, l'utilisation de [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) a en fait dégradé les performances du modèle. Dans votre flux de traitement, comparez les effets de ne pas mettre à l'échelle les données par rapport à l'application de [MinMaxScaler](https://scikit-learn.org/1.5/modules/generated/sklearn.preprocessing.MinMaxScaler.html). Par souci de temps, une seule expérience suffira. Il est important de noter que lorsque la mise à l'échelle est appliquée, une méthode uniforme doit être utilisée pour toutes les colonnes, compte tenu de leur nature homogène.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# apply scaler to features but not target
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train.iloc[:, 1:]), columns=df_train.columns[1:])
df_test_scaled = pd.DataFrame(scaler.transform(df_test.iloc[:, 1:]), columns=df_test.columns[1:])
df_valid_scaled = pd.DataFrame(scaler.transform(df_valid.iloc[:, 1:]), columns=df_valid.columns[1:])

df_train_scaled.head()

> ### Findings of 3. Mise à l'échelle des caractéristiques numériques
>
> As we can see from the results in [5. Développement et évaluation des modèles](#développement-et-évaluation-des-modèles), there is minimal differences between using the unscaled data and data that was scaled using MinMaxScaler. This makes sense because of the way MinMaxScaler works because the feature values are already between 0 and 1, and MinMaxScaler works by scaling the feature values between 0 and 1.

4. **Séparation des cibles et des données** :

    - Dans les fichiers CSV, les cibles et les données sont combinées. Pour préparer nos expériences d'apprentissage automatique, séparez les données d'entraînement $X$ et le vecteur cible $y$ pour chacun des trois ensembles de données.

In [38]:
# Code cell

X_train_scaled, y_train = df_train_scaled, df_train.iloc[:, 0]
X_test_scaled, y_test = df_test_scaled, df_test.iloc[:, 0]
X_valid_scaled, y_valid = df_valid_scaled, df_valid.iloc[:, 0]

X_train_unscaled = df_train.iloc[:, 1:]
X_test_unscaled = df_test.iloc[:, 1:]
X_valid_unscaled = df_valid.iloc[:, 1:]

In [ ]:
X_train_scaled.head()

In [ ]:
y_train.head()

In [ ]:
y_train.value_counts()

### Développement et évaluation des modèles

5. **Développement de modèle** :

    - **Modèle de base** : Implémentez un modèle utilisant le [DummyClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html). Ce modèle ignore les données d'entrée et prédit la classe majoritaire. Un tel modèle est parfois appelé modèle « homme de paille ».

    - **Modèle de référence** : Comme modèle de référence, sélectionnez un des algorithmes d'apprentissage automatique précédemment étudiés : arbres de décision, k-plus proches voisins (KNN) ou régression logistique. Utilisez les paramètres par défaut fournis par scikit-learn pour entraîner chaque modèle en tant que modèle de référence. Pourquoi avez-vous choisi ce classificateur particulier ? Pourquoi pensez-vous qu'il soit approprié pour cette tâche spécifique ?

    - **Modèle de réseau de neurones** : En utilisant [Keras](https://keras.io) et [TensorFlow](https://www.tensorflow.org), construisez un modèle séquentiel comprenant une couche d'entrée, une couche cachée et une couche de sortie. La couche d'entrée doit comporter 462 nœuds, correspondant aux 462 attributs de chaque exemple. La couche cachée doit comprendre 8 nœuds et utiliser la fonction d'activation par défaut. La couche de sortie doit comporter trois nœuds, correspondant aux trois classes : hélice (0), feuillet (1) et enroulement (2). Appliquez la fonction d'activation softmax à la couche de sortie pour que les sorties soient traitées comme des probabilités, avec leur somme égale à 1 pour chaque exemple d'entraînement.

    Nous avons donc trois modèles : de base, de référence et réseau de neurones.

In [ ]:
# Code cell

# Dummy classifier
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train_scaled, y_train)
dummy_clf.score(X_valid_scaled, y_valid)

### Reference model

We choose the reference model to be a Decision Tree because ############

In [ ]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train_unscaled, y_train)
dt_score = dt_clf.score(X_valid_unscaled, y_valid)

dt_clf_scaled = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)
dt_clf_scaled.fit(X_train_scaled, y_train)
dt_scaled_score = dt_clf_scaled.score(X_valid_scaled, y_valid)

print(f"Score without scaling: {dt_score}")
print(f"Score with scaling: {dt_scaled_score}")

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_unscaled, y_train)
knn_score = knn_clf.score(X_valid_unscaled, y_valid)

knn_clf_scaled = KNeighborsClassifier()
knn_clf_scaled.fit(X_train_scaled, y_train)
knn_scaled_score = knn_clf_scaled.score(X_valid_scaled, y_valid)

print(f"Score without scaling: {knn_score}")
print(f"Score with scaling: {knn_scaled_score}")

In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train_unscaled, y_train)
lr_score = lr_clf.score(X_valid_unscaled, y_valid)

lr_clf_scaled = LogisticRegression()
lr_clf_scaled.fit(X_train_scaled, y_train)
lr_scaled_score = lr_clf_scaled.score(X_valid_scaled, y_valid)

print(f"Score without scaling: {lr_score}")
print(f"Score with scaling: {lr_scaled_score}")

In [ ]:
# Keras and tensorflow

import tensorflow as tf

tf.random.set_seed(42)

keras_model_scaled = tf.keras.Sequential()

keras_model_scaled.add(tf.keras.layers.Dense(462, input_dim=462, activation='relu'))
keras_model_scaled.add(tf.keras.layers.Dense(8))
keras_model_scaled.add(tf.keras.layers.Dense(3, activation='softmax'))

keras_model_scaled.summary()

In [ ]:
keras_model_scaled.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = keras_model_scaled.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=25)

In [ ]:
pd.DataFrame(history.history).plot(
    grid=True, figsize=(10, 7), title="Loss and Accuracy", style="--.", ylim=(0, 2)
)
plt.legend(loc="best")
plt.show()

6. **Évaluation des modèles** :

    - Utilisez la validation croisée pour évaluer les performances du modèle de référence. Sélectionnez un petit nombre de plis (*plis*) pour éviter des temps de calcul excessives.

    - **L'entraînement des réseaux de neurones peut être long.** Par conséquent, leurs performances sont généralement évaluées une seule fois en utilisant un ensemble de validation. Assurez-vous de ne pas utiliser l'ensemble de test avant la fin du devoir.

    - Évaluez les modèles en utilisant des métriques telles que la précision, le rappel et le score F1.

In [17]:
# Code cell
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score

def cross_eval_models(model_dict, X_train_, y_train_):
    model_scores = {}
    metrics = {
            "precision": make_scorer(precision_score, average='weighted'),
            "recall": make_scorer(recall_score, average='weighted'),
            "accuracy": make_scorer(accuracy_score),
            "f1": make_scorer(f1_score, average='weighted')
        }
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for name, model in model_dict.items():
        print(f"Model: {name}")
        model_scores[name] = {}
        for metric_name, scorer in metrics.items():
            scores = cross_val_score(model, X_train_, y_train_, scoring=scorer, cv=strat_kfold) 
            print(f"{metric_name}: {scores}")
            print(f"Mean score: {scores.mean():.3f}")
            print(f"StandardDeviation: {scores.std():.3f}")
            model_scores[name][metric_name] = {"scores": scores, "mean": scores.mean(), "std": scores.std()}
        print()
    return model_scores

In [ ]:
model_dict = {"dummy": dummy_clf, "dt": dt_clf_scaled, "knn": knn_clf, "lr": lr_clf}
model_scores = cross_eval_models(model_dict, X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import classification_report

y_pred = keras_model_scaled.predict(X_valid_scaled)

keras_scores = classification_report(y_valid, y_pred.argmax(axis=1))
print(keras_scores)

### Optimisation des hyperparamètres

7. **Modèle de référence :**

    - Pour assurer une comparaison équitable avec notre modèle de référence, nous examinerons comment la variation des hyperparamètres affecte ses performances. Cela évite la conclusion erronée selon laquelle les réseaux de neurones sont intrinsèquement meilleurs, alors qu'en réalité, un ajustement adéquat des hyperparamètres pourrait améliorer les performances du modèle de référence.

    - Concentrez-vous sur les hyperparamètres suivants pour chaque modèle :

        - [DecisionTreeClassifier](https://scikit-learn.org/dev/modules/generated/sklearn.tree.DecisionTreeClassifier.html) : `criterion` et `max_depth`.
  
        - [LogisticRegression](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html) : `penalty`, `max_iter`, et `tol`.
  
        - [KNeighborsClassifier](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) : `n_neighbors` et `weights`.

    - Utilisez une stratégie de recherche par grille ou les méthodes intégrées de scikit-learn [GridSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html) pour évaluer de manière exhaustive toutes les combinaisons de valeurs d'hyperparamètres. La validation croisée doit être utilisée pour évaluer chaque combinaison.

    - Quantifiez les performances de chaque configuration d'hyperparamètres en utilisant des métriques telles que la précision, le rappel et le score F1.

    - Analysez les résultats et fournissez des aperçus sur quelles configurations d'hyperparamètres ont obtenu des performances optimales pour chaque modèle.

In [20]:
# Code cell

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

def grid_search(model, param_grid, X_train_, y_train_, X_test_, y_test_):
    grid = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
    grid.fit(X_train_, y_train_)
    print(f"Best hyperparameters: {grid.best_params_}")
    print(f"Best score: {grid.best_score_}")
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test_)
    
    report = classification_report(y_test_, y_pred)
    print(f"Classification report:\n{report}")
        
    return grid

In [21]:
param_grid_dt = [
    {'max_depth': range(1, 10), # default = None
   'criterion': ["gini", "entropy", "log_loss"] }, # default = gini
]
param_grid_kn = [
  {'n_neighbors': range(3, 10),  # default = 5
   'weights': ["uniform", "distance"]} # default = uniform
]
param_grid_lr = [ 
  {'penalty': ["l2", None], # default = l2 (Note: default solver lbfgs supports only 'l2' or None penalties, so "elasticnet" and "l1" are not supported)
   'max_iter' : [100, 200, 400, 800, 1600], # default = 100
   'tol' : [0.01, 0.001, 0.0001]} # default = 1e-4 
] 

In [ ]:
dt_model = DecisionTreeClassifier()
dt_grid = grid_search(dt_model, param_grid_dt, X_train_scaled, y_train, X_valid_scaled, y_valid)

In [ ]:
knn_model = KNeighborsClassifier()
knn_grid = grid_search(knn_model, param_grid_kn, X_train_scaled, y_train, X_valid_scaled, y_valid)

In [ ]:
lr_model = LogisticRegression()
lr_grid = grid_search(lr_model, param_grid_lr, X_train_scaled, y_train, X_valid_scaled, y_valid)

> ### Observations
> 

8. **Réseau de neurones :**

    Lors de notre exploration et ajustement des réseaux de neurones, nous nous concentrons sur les hyperparamètres suivants :

    - **Une seule couche cachée, en variant le nombre de nœuds**. 

        - Commencez avec un seul nœud dans la couche cachée. Utilisez un graphique pour représenter l'évolution de la perte et de la précision pour les ensembles d'entraînement et de validation, avec l'axe horizontal représentant le nombre d'époques d'entraînement et l'axe vertical représentant la perte et la précision. L'entraînement de ce réseau devrait être relativement rapide, nous allons donc procéder à un entraînement sur 50 époques. Que concluez-vous de l'observation du graphique ? Le réseau sous-apprend-il ou surapprend-il ? Pourquoi ?

In [50]:
# Code cell

def vary_hidden_layer(iteration):
    for num_node, epochs in iteration.items():
        model = keras.Sequential()
        model.add(keras.layers.Dense(462, input_dim=462, activation='relu'))
        model.add(keras.layers.Dense(num_node))
        model.add(keras.layers.Dense(3, activation='softmax'))
        
        model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
        history = model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=50)
        
        pd.DataFrame(history.history).plot(
            grid=True, figsize=(10, 7), title="Loss and Accuracy", style="--.", ylim=(0, 2)
        )
        plt.legend(loc="best")
        plt.title(f"Hidden layer size: {num_node}")
        plt.xlabel("Epochs")
        plt.show()
        

In [ ]:
vary_hidden_layer({1:50})

- Répétez le processus ci-dessus en utilisant 2 et 4 nœuds dans la couche cachée. Utilisez le même type de graphique pour documenter vos observations concernant la perte et la précision.

In [ ]:
# Code cell
iters2_4 = {
    2:50,
    4:50
}
vary_hidden_layer(iters2_4)

- Commencez avec 8 nœuds dans la couche cachée et doublez progressivement le nombre de nœuds jusqu'à ce qu'il dépasse le nombre de nœuds dans la couche d'entrée. Cela donne lieu à sept expériences et graphiques correspondants pour les configurations suivantes : 8, 16, 32, 64, 128, 256 et 512 nœuds. Documentez vos observations tout au long du processus.

In [27]:
# Code cell
iters8_512 = {
    8:50,
    16:50,
    32:50,
    64:50,
    128:50,
    256:50,
    512:50
}
vary_hidden_layer(iters8_512)

- Assurez-vous que le **nombre d'époques d'entraînement** est suffisant pour **observer une augmentation de la perte de validation**. **Conseil** : Lors du développement du modèle, commencez avec un petit nombre d'époques, comme 5 ou 10. Une fois que le modèle semble bien fonctionner, testez avec des valeurs plus importantes, comme 40 ou 80 époques, ce qui s'est avéré raisonnable dans nos tests. En fonction de vos observations, envisagez de mener des expériences supplémentaires, si nécessaire. Combien d'époques ont finalement été nécessaires ?

In [28]:
# Code cell

- **Variation du nombre de couches**.

    - Réalisez des expériences similaires à celles décrites ci-dessus, mais cette fois en faisant varier le nombre de couches de 1 à 4. Documentez vos résultats.

In [29]:
# Code cell

- Combien de nœuds chaque couche devrait-elle contenir ? Testez au moins deux scénarios. Traditionnellement, une stratégie courante consistait à diminuer le nombre de nœuds de la couche d'entrée à la couche de sortie, souvent en divisant par deux, pour créer une structure en pyramide. Cependant, l'expérience récente suggère que le maintien d'un nombre constant de nœuds dans toutes les couches peut également bien fonctionner. Décrivez vos observations. Il est acceptable que les deux stratégies produisent des résultats similaires en termes de performance.

In [30]:
# Code cell

- Sélectionnez un de vos modèles qui illustre le surapprentissage. Dans nos expériences, nous avons facilement construit un modèle atteignant près de 100 % de précision sur les données d'entraînement, sans aucune amélioration similaire sur l'ensemble de validation. Présentez ce réseau de neurones avec ses graphiques de précision et de perte. Expliquez pourquoi vous concluez que le modèle surapprend.

In [31]:
# Code cell

- **Fonction d'activation**.

    - Présentez les résultats pour une des configurations mentionnées ci-dessus en variant la fonction d'activation. Testez au moins `relu` (le paramètre par défaut) et `sigmoid`. Le choix du modèle spécifique, y compris le nombre de couches et de nœuds, est à votre discrétion. Documentez vos observations en conséquence.

In [32]:
# Code cell

- **Régularisation** dans les réseaux de neurones est une technique utilisée pour éviter le surapprentissage.

    - Une technique consiste à ajouter une pénalité à la fonction de perte pour décourager les modèles excessivement complexes. Appliquez une pénalité `l2` à certaines ou à toutes les couches. Soyez prudent, car des pénalités trop agressives se sont révélées problématiques dans nos expériences. Commencez avec la valeur par défaut `l2` de 0.01, puis réduisez-la à 0.001 et à 1e-4. Sélectionnez un modèle spécifique parmi les expériences ci-dessus et présentez un cas où vous avez réussi à réduire le surapprentissage. Incluez une paire de graphiques comparant les résultats avec et sans régularisation. Expliquez votre raisonnement pour conclure que le surapprentissage a été réduit. N'espérez pas éliminer complètement le surapprentissage. Encore une fois, il s'agit d'un ensemble de données difficile à travailler.

In [33]:
# Code cell

- Les couches de **dropout** sont une technique de régularisation dans les réseaux de neurones, où un sous-ensemble aléatoire de neurones est temporairement retiré pendant l'entraînement. Cela aide à éviter le surapprentissage en favorisant la redondance et en améliorant la capacité du réseau à généraliser sur de nouvelles données. Sélectionnez un modèle spécifique parmi les expériences ci-dessus, où vous avez plusieurs couches, et expérimentez l'ajout d'une ou de plusieurs couches de dropout dans votre réseau. Testez deux taux différents, par exemple 0.25 et 0.5. Documentez vos observations.

In [34]:
# Code cell

- Résumez vos expériences en utilisant une représentation graphique telle que la Figure 6.15 [sur cette page](https://egallic.fr/Enseignement/ML/ECB/book/deep-learning.html).

In [35]:
# Code cell

- L'**arrêt anticipé** est une technique de régularisation lors de l'entraînement d'un réseau de neurones, où le processus est interrompu lorsque les performances sur l'ensemble de validation commencent à se dégrader, ce qui empêche le réseau d'apprendre le bruit dans les données d'entraînement. Parmi toutes les expériences menées jusqu'à présent, choisissez **une** configuration (le nombre de couches, le nombre de nœuds, la fonction d'activation, la pénalité L2 et les couches de dropout) qui a donné les meilleures performances. Utilisez un graphique de perte et de précision pour déterminer le nombre optimal d'itérations d'entraînement pour ce réseau. Quel est le nombre optimal d'époques pour cette configuration de réseau et pourquoi ?

In [36]:
# Code cell

### Test

9. **Comparaison des modèles** :

    - Évaluez le modèle de référence sur l'ensemble de test, en utilisant l'ensemble de paramètres optimal identifié lors de la recherche de grille. Appliquez également la meilleure configuration de réseau de neurones à l'ensemble de test.

    - Quantifiez les performances du modèle de référence (meilleure configuration d'hyperparamètres) et de votre réseau de neurones (meilleure configuration) en utilisant des métriques telles que la précision, le rappel et le score F1. Comment ces deux modèles se comparent-ils au modèle de base ?

    - Fournissez des recommandations sur le(s) modèle(s) à choisir pour cette tâche et justifiez vos choix en fonction des résultats de l'analyse.

In [37]:
# Code cell

# Resources